# Importação das bibliotecas

In [1]:
%pip install numpy spacy
%pip install --upgrade numpy spacy
%pip install keras
%pip install tensorflow
%pip install pandas
%pip install scikit-learn
%pip install nltk
%pip install tf2onnx
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata (59 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.66.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 4.25.5 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tf2onnx in c:\users\inteli\appdata\local\programs\python\python312\lib\site-packages (1.16.1)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.66.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importação das bibliotecas
import spacy
import numpy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


# Pré-processamento e Pipeline

## Biblioteca Spacy

In [3]:
# Baixar e carregar o modelo de linguagem para português (uma vez)
# spacy.cli.download("pt_core_news_lg")
nlp = spacy.load("pt_core_news_lg")

In [4]:
# Função para separar as sentenças
def extract_sentences(doc):
    sentences = [sent for sent in doc.sents]  # A função retorna spans (conjuntos de tokens)
    return sentences

# Função para separar em tokens (palavras)
def tokenize(sentences):
    tokens = []
    for sentence in sentences:
        # Retorna tokens diretamente como objetos spaCy
        sentence_tokens = [token for token in sentence]  # Mantém os objetos Token
        tokens.extend(sentence_tokens)
    return tokens

# Função para remover stop words
def extract_stop_words(tokens):
    # Verifica se é um objeto Token e não string
    words_filtered = [token for token in tokens if not token.is_stop]
    return words_filtered

# Função para lematizar as palavras
def extract_lemmatizer(tokens):
    # Lematiza se for objeto Token
    lemmatized_words = [token.lemma_ for token in tokens]
    return lemmatized_words

# Função para remover pontuação
def rem_pontuacao(tokens):
    # Remove pontuação se for objeto Token
    words_no_punctuation = [token for token in tokens if not token.is_punct]
    return words_no_punctuation

# Função para POS tagging
def pos_tagging(tokens):
    # Recebe objetos Token e extrai tags POS e dependências
    pos_tags = [(token.text, token.pos_, token.dep_) for token in tokens]
    return pos_tags


## Teste inicial do pré-processamento

In [5]:
# Teste: Texto simples em português
texto = "me    mostre as    atualizações do dia 11 de agosto    ."
texto_limpo = ' '.join(texto.split())

doc = nlp(texto_limpo.lower())

In [6]:

# Extração de sentenças
resultado = extract_sentences(doc)

# Reprocessar o texto (concatenar as sentenças) para formar um novo 'doc'
tokens = tokenize(resultado)

# POS Tagging
ents = pos_tagging(tokens)

# Remover stop words
resultado = extract_stop_words(tokens)

# Remover pontuações
resultado = rem_pontuacao(resultado)

# Lematização
resultado = extract_lemmatizer(resultado)

print(resultado)
print(ents)

['mostrar', 'atualização', 'dia', '11', 'agosto']
[('me', 'PRON', 'obj'), ('mostre', 'VERB', 'ROOT'), ('as', 'DET', 'det'), ('atualizações', 'NOUN', 'obj'), ('do', 'ADP', 'case'), ('dia', 'NOUN', 'nmod'), ('11', 'NUM', 'nummod'), ('de', 'ADP', 'case'), ('agosto', 'NOUN', 'nmod'), ('.', 'PUNCT', 'punct')]


In [7]:
def to_string(list):
  string = ""
  for word in list:
    string += word + ' '
  return string.strip()

In [8]:
# Text preprocessing function
def preprocess_text(text):
    # Convert the text to lowercase and create a spaCy doc object
    doc = nlp(text.lower())
    
    # Sentence extraction
    resultado = extract_sentences(doc)

    # Tokenization
    tokens = tokenize(resultado)

    # POS tagging (Part of Speech tagging)
    pos_tags = pos_tagging(tokens)

    # Stop words removal (this line is commented out, so stop words won't be removed)
    # tokens = extract_stop_words(tokens)

    # Remove punctuation
    # tokens = rem_pontuacao(tokens)

    # Lemmatization (this line is commented out, so lemmatization won't be applied)
    # tokens = extract_lemmatizer(tokens)

    # Convert the tokens into strings before concatenating them
    tokens = ' '.join([token.text for token in tokens])
    
    # Return the concatenated string of processed tokens
    return tokens


In [9]:
# Text preprocessing function
def preprocess_model(text):
    # Convert the text to lowercase and create a spaCy doc object
    doc = nlp(text.lower())
    
    # Sentence extraction
    resultado = extract_sentences(doc)

    # Tokenization
    tokens = tokenize(resultado)

    # Remove stop words
    tokens = extract_stop_words(tokens)

    # Remove punctuation
    tokens = rem_pontuacao(tokens)

    # Lemmatization (this line is commented out, so lemmatization won't be applied)
    # tokens = extract_lemmatizer(tokens)

    # Convert the tokens into strings before concatenating them
    tokens = ' '.join([token.text for token in tokens])
    
    # Return the concatenated string of processed tokens
    return tokens


In [10]:
texto = "Ai que vontade de viajar senhorr"
preprocess_text(texto)

'ai que vontade de viajar senhorr'

# Treinamento do modelo

In [11]:
from sklearn.svm import LinearSVC

## Base de dados

In [12]:
import pandas as pd

# Dados
data = {
    'frase': [
        "A regulação de mercado busca equilibrar a competição e proteger os consumidores", 
        "O comitê regulador foi criado para garantir a conformidade com as normas setoriais",
        "A auditoria interna identificou falhas nos controles financeiros da empresa", 
        "A fiscalização intensificou suas atividades para prevenir fraudes no setor", 
        "A governança corporativa garante transparência e responsabilidade nas decisões empresariais", 
        "A nova lei de proteção de dados exige que as empresas sejam mais cuidadosas com as informações dos clientes", 
        "A responsabilidade social tornou-se um pilar estratégico nas grandes corporações", 
        "A segurança cibernética é uma prioridade nas empresas de tecnologia financeira", 
        "A regulamentação bancária foi atualizada para enfrentar os desafios das fintechs", 
        "A gestão de riscos é fundamental para mitigar possíveis perdas financeiras", 
        "A conformidade ambiental assegura que as operações da empresa não prejudiquem o meio ambiente", 
        "O ato de diretor estabeleceu novas diretrizes para o funcionamento da empresa", 
        "Um comunicado foi emitido para esclarecer as mudanças na política interna", 
        "O Ato de Diretor definiu novos parâmetros para a política de crédito", 
        "O Ato Normativo Conjunto foi assinado por diferentes órgãos reguladores", 
        "O Ato do Presidente aprovou a reforma nas regulamentações financeiras", 
        "A Carta Circular foi enviada a todas as filiais para informar sobre as mudanças regulatórias", 
        "A Circular nº 345 estipula as novas regras de conduta para os bancos", 
        "Um comunicado conjunto foi emitido pelas agências reguladoras para coordenar a fiscalização", 
        "A Decisão Conjunta entre os reguladores aprovou a fusão das empresas sob determinadas condições", 
        "A Instrução Normativa BCB detalha os procedimentos para a implementação das novas regras de crédito", 
        "A Instrução Normativa Conjunta introduziu novas diretrizes para a proteção de dados financeiros", 
        "A Portaria Conjunta definiu os novos procedimentos de compliance em nível federal", 
        "A Resolução BCB regulamenta o uso de novos instrumentos financeiros no mercado", 
        "A Resolução CMN determina a forma de cálculo das taxas de juros aplicáveis aos empréstimos",
        "O cheiro de café fresco encheu a cozinha naquela manhã fria", 
        "Os pássaros cantavam alegremente ao amanhecer", 
        "Ele passou a tarde inteira lendo um livro de aventuras", 
        "As crianças correram pelo parque, rindo e brincando", 
        "O pôr do sol pintou o céu com tons de laranja e rosa", 
        "Ela decidiu plantar um jardim cheio de flores coloridas", 
        "O som das ondas quebrando na praia era relaxante", 
        "Um vento suave balançava as folhas das árvores no quintal", 
        "Eles se reuniram para um piquenique à beira do rio", 
        "O aroma de pão recém-assado era irresistível", 
        "A montanha coberta de neve parecia um cenário de filme", 
        "O cachorro correu atrás da bola, trazendo de volta com entusiasmo", 
        "Ele passou horas explorando trilhas em uma floresta densa", 
        "O verão trouxe dias longos e ensolarados, perfeitos para passeios", 
        "A noite estrelada parecia um tapete de brilhos no céu", 
        "Eles passaram a tarde jogando cartas em volta da lareira", 
        "As borboletas dançavam entre as flores no jardim", 
        "A brisa do mar refrescava enquanto caminhavam pela praia",
        "O cheiro de terra molhada após a chuva era reconfortante", 
        "Ele passou a manhã fotografando as paisagens ao redor da cidade", 
        "A viagem de trem foi tranquila, com belas paisagens pela janela", 
        "Eles se perderam em uma conversa sobre o universo e as estrelas", 
        "A serenidade do campo proporcionava uma paz indescritível", 
        "Ela passou o dia aprendendo a tocar violão com tutoriais online"
    ],
    'tag': [
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "consulta_normativa", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
        "outros_outros", 
    ]
}

# Criar o DataFrame
df = pd.DataFrame(data)

# Exibir o DataFrame
df.head()


,frase,tag
0,A regulação de mercado busca equilibrar a comp...,consulta_normativa
1,O comitê regulador foi criado para garantir a ...,consulta_normativa
2,A auditoria interna identificou falhas nos con...,consulta_normativa
3,A fiscalização intensificou suas atividades pa...,consulta_normativa
4,A governança corporativa garante transparência...,consulta_normativa


## Aplicando Pré-processamento ao Banco de dados

In [13]:
# Pré-processar as frases usando seu pipeline de pré-processamento
df['frase_processada'] = df['frase'].apply(preprocess_model)
df.head()

,frase,tag,frase_processada
0,A regulação de mercado busca equilibrar a comp...,consulta_normativa,regulação mercado busca equilibrar competição ...
1,O comitê regulador foi criado para garantir a ...,consulta_normativa,comitê regulador criado garantir conformidade ...
2,A auditoria interna identificou falhas nos con...,consulta_normativa,auditoria interna identificou falhas controles...
3,A fiscalização intensificou suas atividades pa...,consulta_normativa,fiscalização intensificou atividades prevenir ...
4,A governança corporativa garante transparência...,consulta_normativa,governança corporativa garante transparência r...


## Divisão de treino e teste dos dados - Treinamento

In [14]:
# Dividir os dados em treino e teste
X = df['frase_processada']  # Features (texto pré-processado)
y = df['tag']  # Labels (intenção)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Vetorizar o texto (TF-IDF após seu pré-processamento)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Treinar o modelo SVM
model = LinearSVC()
model.fit(X_train_vec, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_vec)

## Avaliação das métricas do Modelo

In [15]:
# Avaliar o modelo
print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.75
Relatório de Classificação:
                    precision    recall  f1-score   support

consulta_normativa       0.64      1.00      0.78         9
     outros_outros       1.00      0.55      0.71        11

          accuracy                           0.75        20
         macro avg       0.82      0.77      0.74        20
      weighted avg       0.84      0.75      0.74        20



## Exportação do modelo

In [16]:
import pickle

# Salvar o modelo
with open('modelo_pln.pkl', 'wb') as file:
    pickle.dump(model, file)

In [17]:
# Salvar o vectorizer treinado
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

## Testes e predições

In [18]:
# Lista de novas frases para testar no modelo
novas_frases = [
    # Auditoria Interna
    "Quais são as etapas de uma auditoria interna completa?",
    "Como preparar um plano de auditoria interna eficaz?",
    "Como devo reportar uma não conformidade encontrada durante a auditoria?",
    "Quais são os princípios fundamentais da governança corporativa?",
    "Como podemos melhorar a governança corporativa dentro da nossa organização?",
    "Qual é o papel do conselho de administração na governança corporativa?",
    "Quais são as principais regulamentações de compliance no setor financeiro?",
    "Como garantir que nossa empresa cumpra com as novas leis de privacidade de dados?",
    "Qual é o impacto das normas ISO na certificação de qualidade?",
    "Eu quero todas as normas da BACEM",
    "Eu quero jogar bola",
    "Qual o carro mais rápido do mundo",
]

In [19]:
# Testar o modelo com cada nova frase
for nova_frase in novas_frases:
    # Pré-processar a nova frase
    nova_frase_processada = preprocess_text(nova_frase)

    # nova_frase_processada = to_string(nova_frase_processada)
    
    # Vetorizar a nova frase
    nova_frase_vec = vectorizer.transform([nova_frase_processada])
    
    # Fazer a previsão
    predicao_nova_frase = model.predict(nova_frase_vec)
    
    # Exibir o resultado
    print(f"Frase: {nova_frase}")
    print(f"Intenção prevista: {predicao_nova_frase}")
    print('-' * 50)

Frase: Quais são as etapas de uma auditoria interna completa?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Como preparar um plano de auditoria interna eficaz?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Como devo reportar uma não conformidade encontrada durante a auditoria?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Quais são os princípios fundamentais da governança corporativa?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Como podemos melhorar a governança corporativa dentro da nossa organização?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Qual é o papel do conselho de administração na governança corporativa?
Intenção prevista: ['consulta_normativa']
--------------------------------------------------
Frase: Qua

## Extraindo entidades

In [20]:
import re
from datetime import datetime, timedelta

In [21]:
# Função para processar datas
def processar_data(expressao_data):
    if expressao_data is None:
        return None

    hoje = datetime.today()

    if expressao_data.lower() == "semana passada" or "semana passado":
        data_inicial = hoje - timedelta(days=hoje.weekday() + 7)
        data_final = data_inicial + timedelta(days=6)
        return f"{data_inicial.date()} até {data_final.date()}"

    elif expressao_data.lower() == "mês passado":
        primeiro_dia_mes = hoje.replace(day=1)
        ultimo_dia_mes_passado = primeiro_dia_mes - timedelta(days=1)
        primeiro_dia_mes_passado = ultimo_dia_mes_passado.replace(day=1)
        return f"{primeiro_dia_mes_passado.date()} até {ultimo_dia_mes_passado.date()}"

    elif expressao_data.lower() == "ano passado":
        ano_passado = hoje.year - 1
        return f"01/01/{ano_passado} até 31/12/{ano_passado}"

    elif expressao_data.lower() == "hoje":
        return hoje.strftime("%d/%m/%Y")

    elif expressao_data.lower() == "ontem":
        ontem = hoje - timedelta(days=1)
        return ontem.strftime("%d/%m/%Y")

    elif expressao_data.lower() == "próxima semana":
        inicio_semana_proxima = hoje + timedelta(days=(7 - hoje.weekday()))
        fim_semana_proxima = inicio_semana_proxima + timedelta(days=6)
        return f"{inicio_semana_proxima.date()} até {fim_semana_proxima.date()}"

    # Captura padrões como "julho de 2023" ou "dezembro de 2022"
    elif re.match(r"(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de \d{4}", expressao_data.lower()):
        # Mapeia os meses para números
        meses = {
            "janeiro": 1, "fevereiro": 2, "março": 3, "abril": 4, "maio": 5, "junho": 6,
            "julho": 7, "agosto": 8, "setembro": 9, "outubro": 10, "novembro": 11, "dezembro": 12
        }
        partes = expressao_data.lower().split(" de ")
        mes = meses[partes[0]]
        ano = int(partes[1])
        primeiro_dia = datetime(ano, mes, 1)
        if mes == 12:
            ultimo_dia = datetime(ano, mes, 31)
        else:
            ultimo_dia = datetime(ano, mes + 1, 1) - timedelta(days=1)
        return f"{primeiro_dia.date()} até {ultimo_dia.date()}"

    elif re.match(r"\d{1,2}/\d{1,2}/\d{4}", expressao_data):
        return expressao_data  # Retorna a data no formato dd/mm/yyyy

    return expressao_data

In [54]:
# Função para enriquecer ainda mais a extração de entidades com regex
def extrair_entidades_regex(texto):
    entidades = {
        'regulator': 'BACEN',
        'type': None,
        'number':None,
        'topic':None,
        'date': None,
        'organization': None,
        'tag': None,
    }

    # Padrões regex ainda mais ricos
    padrao_regulator = r"\b(BACEM)\b"

    padrao_type = r"\b(Ato de Diretor|Ato Normativo Conjunto|Ato do Presidente|Carta Circular|Circular|Comunicado|Comunicado Conjunto|Decisão Conjunta|Instrução Normativa BCB|Instrução Normativa Conjunta|Portaria Conjunta|Resolução BCB|Resolução CMN|Resolução Conjunta|Resolução Coremec)\b"

    #ANALISAR LÓGICA PARA OS TÓPICOS
    padrao_topic = r"\b(Divulga a Taxa Básica Financeira)\b"

    padrao_organization = r"\b(DEMAB)\b"

    #Adicionar lógica para buscar normas revogadas

    padrao_tag = r"\b(Taxa Básica Financeira(TBF)|Operações do Mercado Aberto)\b"

    padrao_date = r"\b(de hoje|ultimo dia|semana passada|semana passdo|mês passado|ano passado|hoje|ontem|próxima semana|dezembro de \d{4}|janeiro de \d{4}|fevereiro de \d{4}|março de \d{4}|abril de \d{4}|maio de \d{4}|junho de \d{4}|julho de \d{4}|agosto de \d{4}|setembro de \d{4}|outubro de \d{4}|novembro de \d{4}|\d{1,2}/\d{1,2}/\d{4})\b"

    padrao_number = r"\b\d+\b"

    regulator = re.search(padrao_regulator, texto, re.IGNORECASE)
    if regulator:
        entidades['regulator'] = regulator.group(0)

    type = re.search(padrao_type, texto, re.IGNORECASE)
    if type:
        entidades['type'] = type.group(0)  # Mantém o formato original do texto

    number = re.search(padrao_number, texto, re.IGNORECASE)
    if number:
        entidades['number'] = number.group(0)

    topic = re.search(padrao_topic, texto, re.IGNORECASE)
    if topic:
        entidades['topic'] = topic.group(0)

    date = re.search(padrao_date, texto, re.IGNORECASE)
    if date:
        entidades['date'] = processar_data(date.group(0))

    organization = re.search(padrao_organization, texto, re.IGNORECASE)
    if organization:
        entidades['organization'] = organization.group(0)

    tag = re.search(padrao_tag, texto, re.IGNORECASE)
    if tag:
        entidades['tag'] = tag.group(0)

    return entidades

In [23]:
# Função para identificar entidades
def entidades(nova_frase_processada):
    print(nova_frase_processada)
    entidades = extrair_entidades_regex(nova_frase_processada)  # Extrai as entidades usando regex
    print(entidades)
    return entidades

# Turbinando o Pré-processamento com IA Generativa - Gemini

In [24]:
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.2 which is incompatible.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 5.28.2 which is incompatible.


In [39]:
lista_tipos = [
    "BACEM",
    "DEMAB"
    "Taxa Básica Financeira(TBF)",
    "Operações do Mercado Aberto",
    "Ato de Diretor", "Ato Normativo Conjunto", "Ato do Presidente", 
    "Carta Circular", "Circular", "Comunicado", "Comunicado Conjunto", 
    "Decisão Conjunta", "Instrução Normativa BCB", "Instrução Normativa Conjunta", 
    "Portaria Conjunta", "Resolução BCB", "Resolução CMN", "Resolução Conjunta", 
    "Resolução Coremec", "Regulação Financeira", "Fiscalização financeira", 
    "Governança regulatoria", "Fiscalização Interna", "Circular", "Governança", 
    "Financial Services"
    
]

In [40]:
context = f"""
Você é um especialista em identificar a inteção do usuário e corrigir texto.

lista com as palavaras corretas que temos no nosso banco: {lista_tipos}

Exemplo de Input do usuário:
Eu quero o comunicadu número quarenta e dois mill

Seu output:
Eu quero o comunicado n° 42000

Siga essas instruções a risca:
- Retorne apenas a frase corrigida sem nenhuma informação adicional
- Nuemro com ponto, retorne apenas o numero (42.031 -> 42031)
- Não remova informações do usuário. Apenas corrija o português e analise se ele quer usar alguma das palavras do nosso banco de palavras

"""

In [43]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# Create the model
generation_config = {
  "temperature": 0.8,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

gemini = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  system_instruction=context,
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = gemini.start_chat(
  history=[
  ]
)

def gemini_chat(input):
  response = gemini.generate_content(f"""Pergunta: {input}""")
  print(response.text)
  return response.text

## Testes Gemini

In [47]:
nova_frase = gemini_chat('comunicadu numero quareta e dois mil e 171 que divulga a taxa básia financeira da BACEN da organização DEMAB da tag Operações do dia de hoje ')

Comunicado nº 42171 que divulga a Taxa Básica Financeira (TBF) do BACEN da organização DEMAB da tag Operações do Mercado Aberto de hoje 



## Testando extração de informações

In [55]:
# Testar o modelo com cada nova frase
# for nova_frase in novas_frases:
# Pré-processar a nova frase
# nova_frase_processada = preprocess_text(nova_frase)
# Vetorizar a nova frase
nova_frase_vec = vectorizer.transform([nova_frase])
# Fazer a previsão
predicao_nova_frase = model.predict(nova_frase_vec)

if predicao_nova_frase == 'consulta_normativa':
    ent = entidades(nova_frase)        
    print('-' * 50)

else:
    print("Perdão, não posso responder a esse tipo de pergunta.")
    print('-' * 50)

Comunicado nº 42171 que divulga a Taxa Básica Financeira (TBF) do BACEN da organização DEMAB da tag Operações do Mercado Aberto de hoje 

{'regulator': 'BACEN', 'type': 'Comunicado', 'number': '42171', 'topic': 'divulga a Taxa Básica Financeira', 'date': '2024-09-16 até 2024-09-22', 'organization': 'DEMAB', 'tag': 'Operações do Mercado Aberto'}
--------------------------------------------------


# Construção do SQL de consulta

In [51]:
def construir_query(dados):
    # Começa a construção da query
    query = f"""    
    SELECT
        regulation.*,
        regulator.name AS regulator_name,
        regulator.scrapingurl AS regulator_scrapingul,
        organization.name AS organization_name,
        documenttype.name AS documenttype_name,
        tag.name AS tag_name
    FROM
        regulation
    INNER JOIN
        regulator ON regulation.regulatorid = regulator.id
    INNER JOIN
        documenttype ON regulation.documenttypeid = documenttype.id
    INNER JOIN
        organization ON regulation.organizationid = organization.id
    INNER join
        regulationtags ON regulation.id = regulationtags.regulationid
    inner join
        tag ON tag.id = regulationtags.tagid

    WHERE
    
    """
    
    # Lista para armazenar condições
    condicoes = []
    
    # Verifica cada campo e adiciona a condição se não for None
    if dados['regulator'] is not None:
        condicoes.append(f" regulator.name ILIKE '{dados['regulator']}'")
    
    if dados['type'] is not None:
        condicoes.append(f" documenttype.name ILIKE '{dados['type']}'")

    if dados['number'] is not None:
        condicoes.append(f" regulation.documentnumber = '{dados['number']}'")

    # if dados['topic'] is not None:
    #     condicoes.append(f" topic ILIKE = '{dados['topic']}'")

    # if dados['date'] is not None:
    #     condicoes.append(f" publicationdate ILIKE '{dados['date']}'")

    if dados['organization'] is not None:
        condicoes.append(f" organization.name ILIKE '{dados['organization']}'")
    
    if dados['tag'] is not None:
        condicoes.append(f" tag.name ILIKE '{dados['tag']}'")

    if condicoes:
        query += " AND".join(condicoes)
    else:
        query = query.replace(" WHERE", "")  # Remove o WHERE se não houver condições
    
    return query

In [52]:
# Exemplo de uso:
sql_query = construir_query(ent)
print(sql_query)

    
    SELECT
        regulation.*,
        regulator.name AS regulator_name,
        regulator.scrapingurl AS regulator_scrapingul,
        organization.name AS organization_name,
        documenttype.name AS documenttype_name,
        tag.name AS tag_name
    FROM
        regulation
    INNER JOIN
        regulator ON regulation.regulatorid = regulator.id
    INNER JOIN
        documenttype ON regulation.documenttypeid = documenttype.id
    INNER JOIN
        organization ON regulation.organizationid = organization.id
    INNER join
        regulationtags ON regulation.id = regulationtags.regulationid
    inner join
        tag ON tag.id = regulationtags.tagid

    WHERE
    
     regulator.name ILIKE 'BACEN' AND documenttype.name ILIKE 'Comunicado' AND regulation.documentnumber = '42171' AND organization.name ILIKE 'DEMAB'


## Explicabilidade - Bag of Words

A explicabilidade serve para justificar, minimamente, o porquê nosso modelo está tomando certas decisões.

In [32]:
import re
import numpy as np

In [33]:
# Função que faz a tokenização das palavras
def tokenize_bow(sentences):
    vocab = set()
    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            if not token.is_stop and not token.is_punct:
                vocab.add(token.text.lower())
    return sorted(vocab)

def extract_stop_words_bow(sentence):
    doc = nlp(sentence)
    return [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]

def generate_bow_bow(allsentences):
    vocab = tokenize_bow(allsentences)
    print("Word List for Document \n{0} \n".format(vocab))

    for sentence in allsentences:
        words = extract_stop_words_bow(sentence)
        bag_vector = np.zeros(len(vocab))
        for w in words:
            if w in vocab:
                index = vocab.index(w)
                bag_vector[index] += 1

        print("{0} \n{1}\n".format(sentence, np.array(bag_vector)))

In [34]:
generate_bow_bow(novas_frases)

Word List for Document 
['administração', 'auditoria', 'bacem', 'bola', 'carro', 'certificação', 'completa', 'compliance', 'conformidade', 'corporativa', 'cumpra', 'dados', 'devo', 'durante', 'eficaz', 'empresa', 'encontrada', 'etapas', 'financeiro', 'fundamentais', 'garantir', 'governança', 'impacto', 'interna', 'iso', 'jogar', 'leis', 'melhorar', 'mundo', 'normas', 'organização', 'papel', 'plano', 'podemos', 'preparar', 'principais', 'princípios', 'privacidade', 'qualidade', 'regulamentações', 'reportar', 'rápido', 'setor'] 

Quais são as etapas de uma auditoria interna completa? 
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Como preparar um plano de auditoria interna eficaz? 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]

Como devo reportar uma não conformidade encontrada durante a auditoria? 
[0. 1. 0. 0. 0. 0.

# Normas Associadas

In [2]:
associate_types = [
    "Ato de Diretor", "Ato Normativo Conjunto", "Ato do Presidente", 
    "Carta Circular", "Circular", "Comunicado", "Comunicado Conjunto", 
    "Decisão Conjunta", "Instrução Normativa BCB", "Instrução Normativa Conjunta", 
    "Portaria Conjunta", "Resolução BCB", "Resolução CMN", "Resolução Conjunta", 
    "Resolução Coremec"
    
]

In [3]:
associate_context = f"""
    Você é um especialista em identificar normas e regulamentos citados nos textos.

    lista com as palavras-chaves de tipos de regulamentos da BACEN: {associate_types}

    Exemplo de texto da regulamentação:
    O Banco Central do Brasil, com base no disposto na Resolução BCB nº 75, de 23 de fevereiro de 2021, torna público que, das 12:00 às 12:30 horas do dia 2 de outubro de 2024, acolherá propostas das instituições financeiras participantes do módulo Ofpub para a realização de operações de venda de títulos públicos com compromisso de revenda assumido pela instituição financeira compradora, admitida a livre movimentação dos títulos, com as seguintes características:

    Seu output:
    backlinks: {
        [
            {
                "documenttype": "Resolução BCB",
                "documentnumber": 75,
                "publicationdate": "23/02/2021"
            }
        ]
    }

    Siga essas instruções a risca:
    - Retorne apenas a lista de normas e regulamentos citados no texto
    - Retorne apenas regulamentos cujo tipo esteja na lista de tipos de regulamentos da BACEN
    - Nuemro com ponto, retorne apenas o numero (42.031 -> 42031)
    - Não remova informações do texto, principalmente tipo de regulamento e numeração. Apenas identifique as normas e regulamentos citados

"""

In [4]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# Create the model
associate_generation_config = {
  "temperature": 0.8,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

associate_gemini = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  system_instruction=associate_context,
  generation_config=associate_generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

associate_chat_session = associate_gemini.start_chat(
  history=[
  ]
)

def associate_gemini_chat(input):
  response = associate_gemini.generate_content(f"""Regulação: {input}""")
  print(response.text)
  return response.text

c:\Users\Inteli\Documents\GitHub\2024-2A-T09-ES07-G04\src\backend\pln\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
regulation = associate_gemini_chat("""
 O Banco Central do Brasil, tendo em vista o disposto na Resolução CMN nº 2.939, de 26 de março de 2002, e na Resolução BCB nº 76, de 23 de fevereiro de 2021, torna público que, das 11:30 às 11:40 horas do dia 11 de setembro de 2024, acolherá propostas das instituições financeiras participantes do módulo Oferta Pública (Ofpub) para a realização de operações de swap a serem registradas na B3 - Brasil, Bolsa, Balcão S.A., nos termos do "Contrato de Swap Cambial com Ajuste Periódico Baseado em Operações Compromissadas de Um Dia - SCS" daquela bolsa, com as seguintes características:
""")

backlinks: [{'documenttype': 'Resolução CMN', 'documentnumber': 2939, 'publicationdate': '26/03/2002'}, {'documenttype': 'Resolução BCB', 'documentnumber': 76, 'publicationdate': '23/02/2021'}]


In [6]:
def construir_query_backlink(lista_dados):
    # Começa a construção da query
    query = f"""    
    SELECT
        regulation.id,
        documenttype.name,
        regulation.documentnumber
    FROM
        regulation
        JOIN documenttype on documenttype.id = regulation.documenttypeid
    WHERE
    """
    
    # Lista para armazenar condições de cada regulamento
    condicoes = []
    
    # Loop sobre cada item da lista de dados
    for dados in lista_dados:
        # Verifica se os campos não são None e monta a condição para cada regulamento
        condicao = []
        
        if 'documenttype' in dados and dados['documenttype'] is not None:
            condicao.append(f"documenttype.name ILIKE '{dados['documenttype']}'")
        
        if 'documentnumber' in dados and dados['documentnumber'] is not None:
            condicao.append(f"regulation.documentnumber = {dados['documentnumber']}")
        
        if 'publicationdate' in dados and dados['publicationdate'] is not None:
            condicao.append(f"regulation.publicationdate = '{dados['publicationdate']}'")
        

        if condicao:
            condicoes.append(f"({' AND '.join(condicao)})")
    
    if condicoes:
        query += " OR ".join(condicoes)
    else:
        query = None
    
    return query

In [7]:
sql_query = construir_query_backlink([{'documenttype': 'Resolução CMN', 'documentnumber': 2939, 'publicationdate': '26/03/2002'}, {'documenttype': 'Resolução BCB', 'documentnumber': 76, 'publicationdate': '23/02/2021'}])
print(sql_query)

    
    SELECT
        regulation.id,
        documenttype.name,
        regulation.documentnumber
    FROM
        regulation
        JOIN documenttype on documenttype.id = regulation.documenttypeid
    WHERE
    (documenttype.name ILIKE 'Resolução CMN' AND regulation.documentnumber = 2939 AND regulation.publicationdate = '26/03/2002') OR (documenttype.name ILIKE 'Resolução BCB' AND regulation.documentnumber = 76 AND regulation.publicationdate = '23/02/2021')
